In [1]:
!pip install ipdb

In [2]:
import pandas as pd
import numpy as np
import os
import torch
from tqdm import tqdm
from models import *
from dataSet.reader import *
from dataSet.transform import *
import os
import shutil
import ipdb
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
num_TTA = 2

import ipdb

def train_collate(batch):
#     ipdb.set_trace()
    batch_size = len(batch)
    images = []
    labels = []
    names = []
    for b in range(batch_size):
        if batch[b][0] is None:
            continue
        else:
            images.extend(batch[b][0])
            labels.append(batch[b][1])
            names.append(batch[b][2])
    images = torch.stack(images, 0)
    labels = torch.from_numpy(np.array(labels))
    return images, labels, names

def transform(image, mask):
    raw_iamge = cv2.resize(image, (512, 256))
    raw_mask = cv2.resize(mask, (512, 256))
    raw_mask = raw_mask[:, :, None]
    raw_iamge = np.concatenate([raw_iamge, raw_mask], 2)
    images = []

    image = raw_iamge.copy()
    image = np.transpose(image, (2, 0, 1))
    image = image.copy().astype(np.float)
    image = torch.from_numpy(image).div(255).float()
    images.append(image)

    image = raw_iamge.copy()
    image = np.fliplr(image)
    image = np.transpose(image, (2, 0, 1))
    image = image.copy().astype(np.float)
    image = torch.from_numpy(image).div(255).float()
    images.append(image)

    return images

In [3]:
mode = 'data'
names_test = os.listdir(f'./WC_input/{mode}')
sample_submission = pd.read_csv('./WC_input/sample_submission.csv', header=None)
test_imgs = sample_submission.iloc[:, 0].tolist()
batch_size = 80
dst_test = WhaleTestDataset(names_test, mode=mode, transform=transform)
dataloader_test = DataLoader(dst_test, batch_size=batch_size, num_workers=0, collate_fn=train_collate)

loading bbox...


In [4]:
for data in tqdm(dataloader_test):
    images, labels, names = data
    break

  0%|          | 0/67 [00:03<?, ?it/s]


In [5]:
labels.shape

torch.Size([80, 18])

In [6]:
len(names)

80

In [7]:
images.shape

torch.Size([160, 4, 256, 512])

In [8]:
len(names)

80